In [ ]:
# Install required packages
!pip install sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requi

In [ ]:
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load your fine-tuned model
model_path = "/content/drive/MyDrive/ssmt_project/Models/mbart-en-te-final"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50TokenizerFast.from_pretrained(model_path)

# Set language codes
tokenizer.src_lang = "en_XX"
model.config.forced_bos_token_id = tokenizer.lang_code_to_id["te_IN"]

# Load the new data to translate
input_csv_path = "/content/drive/MyDrive/ssmt_project/asr_output_all.csv"
df = pd.read_csv(input_csv_path)

# Translation function
def translate(text_list, batch_size=8):
    translations = []
    model.eval()
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(model.device)
        with torch.no_grad():
            generated_tokens = model.generate(**inputs, max_length=128, num_beams=5)
        outputs = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        translations.extend(outputs)
    return translations

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Translate both columns
df["actual_en_translated"] = translate(df["actual_en"].tolist())
df["asr_decoded_translated"] = translate(df["asr_decoded"].tolist())

# Save the output to a new CSV
output_csv_path = "/content/drive/MyDrive/ssmt_project/eval_translated.csv"
df.to_csv(output_csv_path, index=False)

print(f"Translated file saved at: {output_csv_path}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1581: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Translated file saved at: /content/drive/MyDrive/ssmt_project/eval_translated.csv
